In [1]:
import os
import pickle
from glob import glob

from tqdm import tqdm

In [2]:
action_list = list("()+-*/=") + ["sq", "sqrt"] + [f"c{c}" for c in range(13)]

def state_to_exp(state):
    return list(map(lambda x: action_list[x], state))

def exp_to_state(exp):
    return list(map(lambda x: action_list.index(x), exp))

## AUCs to seen

In [ ]:
seen_path = "cache/cache-cloud/seen.pkl"
seen_set = set()
if os.path.exists(seen_path):
    with open(seen_path, "rb") as fp:
        seen_set = pickle.load(seen_path)

auc_paths = ["logs/logs-cloud/aucs.txt", "logs/logs-cloud-1/aucs-1.txt", "logs/logs-cloud-2/aucs.txt"]

tot_count = 0
for path in auc_paths:
    with open(path, "r") as fp:
        lines = fp.read().split("\n")[:-1]
    tot_count += len(lines)
    for line in lines:
        exp = line.split(" ", 2)[-1]
        seen_set.add(exp)

print(len(seen_set), tot_count)
with open(seen_path, "wb") as fp:
    pickle.dump(seen_set, fp)


## AUCs to Cache

In [ ]:
with open("/home/dilith/Projects/IndexRL/logs/logs-cloud/aucs.txt", "r") as fp:
    lines = fp.read().split("\n")[:-1]

data_buffer = []

for line in lines:
    elems = line.split()
    expression = eval("".join(elems[2:]))
    data_buffer.append((exp_to_state(expression), float(elems[1])))

with open(f"/home/dilith/Projects/IndexRL/cache/cache-cloud/data_buffer_0.pkl", "wb") as fp:
    pickle.dump(data_buffer, fp)


## Update Cache Rewards

In [ ]:
from training_reward import get_training_based_reward


cache_dir = "cache/cache-cloud-rew"
image_dir = "/home/dilith/Projects/SerpSeg/data/subscenes/"
mask_dir = "/home/dilith/Projects/SerpSeg/data/masks/"
cache_paths = glob(os.path.join(cache_dir, "data_buffer*.pkl"))

seen = set()
new_cache = []
for path in cache_paths:
    with open(path, "rb") as fp:
        cache = pickle.load(fp)
    for state, _ in tqdm(cache):
        exp = state_to_exp(state)
        if len(exp) < 3:
            continue
        reward = get_training_based_reward(exp, image_dir, mask_dir)
        if str(exp) in seen:
            continue
        new_cache.append((state, reward))
        seen.add(str(exp))

with open(os.path.join(cache_dir, "data_buffer_0.pkl"), "wb") as fp:
    pickle.dump(new_cache, fp)

with open(os.path.join(cache_dir, "seen.pkl"), "wb") as fp:
    pickle.dump(seen, fp)


## Cache to AUCs

In [7]:
cache_path = "cache/cache-cloud-rew/data_buffer_00.pkl"
auc_path = "logs/logs-cloud-rew/aucs.txt"

with open(cache_path, 'rb') as fp:
    cache = pickle.load(fp)
    
with open(auc_path, 'w') as fp:
    for state, reward in cache:
        fp.write(f"{reward} {state_to_exp(state)}\n")

## Cache exp to state

In [5]:
cache_path = "cache/cache-cloud-rew/data_buffer_0.pkl"
new_cache_path = "cache/cache-cloud-rew/data_buffer_00.pkl"

with open(cache_path, 'rb') as fp:
    cache = pickle.load(fp)

new_cache = []
for exp, reward in tqdm(cache):
    new_cache.append((exp_to_state(exp), reward))

with open(new_cache_path, 'wb') as fp:
    pickle.dump(new_cache, fp)

100%|██████████| 165/165 [00:00<00:00, 63329.08it/s]
